In [1]:
!pip install -q transformers accelerate flash_attn
!apt-get update
!apt-get -y install libgl1-mesa-glx
!apt-get -y install libglib2.0-0
!pip install opencv-python
!pip install bitsandbytes


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libglib2.0-0 is already the newest version (2.72.4-0ubuntu2.4).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.

[notice] A new release of pip is a

In [2]:
from transformers import LlavaProcessor, LlavaForConditionalGeneration
import torch
model_id = "llava-hf/llava-interleave-qwen-0.5b-hf"

processor = LlavaProcessor.from_pretrained(model_id)

model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16,load_in_4bit=True)
model.to("cuda") # can also be xpu, mps, npu etc. depending on your hardware accelerator

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


LlavaForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(729, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-25): 26 x SiglipEncoderLayer(
            (self_attn): SiglipSdpaAttention(
              (k_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear

In [3]:
import uuid
import requests
import cv2
from PIL import Image

def replace_video_with_images(text, frames):
  return text.replace("<video>", "<image>" * frames)

import cv2
from PIL import Image
import numpy as np

def video_to_pil_frames(video_path):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total Frames:{total_frames}")
    frames = []
    for i in range(total_frames):
        ret, frame = video.read()
        if not ret:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        pil_image = Image.fromarray(frame_rgb)
        frames.append(pil_image)
    
    video.release()
    return frames


In [4]:
!pwd

/workspace


In [5]:
frames = video_to_pil_frames("Shot_Detection/movies_to_segment_1/The Wild Bunch (1969)Autoshot/The Wild Bunch (1969)_part_4.mp4")

Total Frames:128


In [6]:
frames

[<PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=720x480>,
 <PIL.Image.Image image mode=RGB size=72

In [7]:
user_prompt = "What is this video about?"
toks = "<image>" * len(frames[:6])
prompt = "<|im_start|>user"+ toks + f"\n{user_prompt}<|im_end|><|im_start|>assistant"
inputs = processor(text=prompt, images=frames[:6], return_tensors="pt").to(model.device, model.dtype)

In [15]:
output = model.generate(**inputs, max_new_tokens=512, do_sample=False)
print(processor.decode(output[0], skip_special_tokens=True))

# The first cat is shown in a relaxed state, with its eyes closed and a content expression, while the second cat is shown in a more active state, with its mouth open wide, possibly in a yawn or a vocalization.



Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


user
What is this video about?assistant The video appears to be a scene from a film or a television show, featuring a group of men riding horses in a desert-like setting. The men are dressed in traditional military uniforms, which are typical of the uniforms of the American military. The horses are brown and have a saddle on their back, which is typical of the saddles used by soldiers in the military. The men are moving in a coordinated manner, suggesting a sense of purpose or purposeful movement. The overall color palette of the scene is dominated by the brown and white tones of the men's uniforms, which contrast with the dusty and open landscape. The lighting is natural and diffused, with a soft glow that suggests an outdoor setting.


In [16]:
output

tensor([[151644,    872, 151646,  ...,   6243,     13, 151645]],
       device='cuda:0')